# Load Data

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from result_saver import SaverProvider
provider = SaverProvider()

In [3]:
import numpy as np
from Scratch import metadata_loader

DEVICE = 'ibm_sherbrooke'

md = metadata_loader(_extract=True, _drop_inutile=True)
md = md[md["job_status"] == "JobStatus.DONE"]
md = md[md["notebook_name"] == "bigger_rep_codes"]
max_distance = int(max(md.distance))
max_distance = 10
md = md[md["distance"] == max_distance]
md = md[md["backend_name"]==DEVICE]

md = md[:2]

md

,creation_date,notebook_name,backend_name,job_id,tags,shots,tags_xp,sampled_state,num_qubits,job_status,extra,optimization_level,code,distance,rounds,logical,layout,descr
243,2023-10-30 09:43:24.564124+01:00,bigger_rep_codes,ibm_sherbrooke,cmzpsb5daqbg008sjvb0,[],10000.0,NaN,NaN,NaN,JobStatus.DONE,NaN,NaN,RepetitionCodeCircuit,10.0,10,1,_is_hex=True,Run bigger Repetition codes v4: new distances ...
242,2023-10-30 09:43:20.984078+01:00,bigger_rep_codes,ibm_sherbrooke,cmzpsa5vayrg008cpk30,[],10000.0,NaN,NaN,NaN,JobStatus.DONE,NaN,NaN,RepetitionCodeCircuit,10.0,10,0,_is_hex=True,Run bigger Repetition codes v4: new distances ...


In [7]:
memories = {}
for job_id, logical in zip(md.job_id, md.logical):
    mmr_name = f"mmr_log_{logical}"
    memories[mmr_name] = provider.retrieve_job(job_id).result().get_memory()

memory = memories["mmr_log_1"][:]

# UF Decoder

In [8]:
import pickle
from qiskit_qec.circuits import RepetitionCodeCircuit
from soft_info import get_repcode_layout, get_KDEs, get_counts

layout = get_repcode_layout(distance=max_distance, backend=provider.get_backend("ibm_nazca"), _is_hex=True)
kde_dict, scaler_dict = get_KDEs(provider, DEVICE, layout, bandwidths=0.2, plot=False)
code = RepetitionCodeCircuit(max_distance, max_distance)


In [12]:
from qiskit_qec.decoders.hdrg_decoders import UnionFindDecoder

dcd_pkl_str = f"dist_{max_distance}_decoder.pkl"

try:
    with open(f"dist_{max_distance}_decoder.pkl", 'rb') as f:
        decoder = pickle.load(f)

except FileNotFoundError:
    print("decoder not found, creating a new one...")
    code = RepetitionCodeCircuit(max_distance, max_distance)
    decoder = UnionFindDecoder(code)    

    print("saving the decoder...")
    with open(dcd_pkl_str, 'wb') as f:
        pickle.dump(decoder, f)

In [14]:
from soft_info import soft_reweight, get_counts, rx_draw_2D
from tqdm import tqdm

VERBOSE = True

logical_counts = {'0':0, '1':0}
err = 0
for shot in tqdm(range(len(memory))):
    
    with open(dcd_pkl_str, 'rb') as f: # Reload the decoder each time to reset the graph
        decoder = pickle.load(f)

    IQ_data = memory[shot]

    counts = get_counts([IQ_data], kde_dict, scaler_dict, layout, max_distance, verbose=False)
    count_key = next(iter(counts.keys()))
    
    logical, flipped_qubit_dict = decoder.process(count_key, _return_err_str = True)
    
    if VERBOSE and logical[0] == 0:
        print("\nmeas_str:", count_key)
        print("flipped_qubit_dict", flipped_qubit_dict)
        print("logical:", logical, "\n")
            
    if logical[0] == 0:
        err += 1
        print("nb of errs:", err)
    logical_counts[f"{logical[0]}"] += 1

print( "\nLogical_counts:", logical_counts)


  7%|▋         | 672/10000 [00:56<13:10, 11.80it/s]


meas_str: 0000111111 000101001 000001000 000101001 100000111 010101010 000000110 000000000 000000110 000000000 000000100
flipped_qubit_dict {'Cluster_0': [2], 'Cluster_1': [4, 5], 'Cluster_2': [8], 'Cluster_3': [1, 1], 'Cluster_4': [8], 'Cluster_5': [1], 'Cluster_6': [3], 'Cluster_7': [0]}
logical: [0] 

nb of errs: 1


  7%|▋         | 712/10000 [01:00<13:35, 11.39it/s]


meas_str: 1110111110 010011111 011110000 000001011 001101000 000001011 011100000 000001001 011000000 000011000 000000000
flipped_qubit_dict {'Cluster_0': [4], 'Cluster_1': [7], 'Cluster_2': [1], 'Cluster_3': [5, 8, 9], 'Cluster_4': [], 'Cluster_5': [2, 3]}
logical: [0] 

nb of errs: 2


  8%|▊         | 840/10000 [01:10<13:20, 11.45it/s]

...


meas_str: 1111110111 001010000 001011100 010000000 001010101 010000000 001010110 000000000 000010100 000000000 000000110
flipped_qubit_dict {'Cluster_0': [2, 3, 2, 4, 6, 3, 8, 9, 5, 2, 4, 5, 5], 'Cluster_1': [1], 'Cluster_2': [0], 'Cluster_3': [4], 'Cluster_4': [7]}
logical: [0] 

nb of errs: 49


 95%|█████████▍| 9490/10000 [13:13<00:42, 12.14it/s]


meas_str: 1000101101 101001011 001111000 101100011 001011000 001000011 001111000 001000011 011000000 001100001 001000001
flipped_qubit_dict {'Cluster_0': [], 'Cluster_1': [4, 6, 5, 7, 3, 2, 4, 4, 5, 5, 0, 6, 7, 4], 'Cluster_2': [9], 'Cluster_3': []}
logical: [0] 

nb of errs: 50


100%|██████████| 10000/10000 [13:56<00:00, 11.95it/s]


Logical_counts: {'0': 50, '1': 9950}


# PyMatching

In [9]:
import stim
import pymatching

d = max_distance
T = max_distance

circuit = stim.Circuit.generated("repetition_code:memory",
                                 distance=d,
                                 rounds=T,
                                 after_clifford_depolarization=0.1)

model = circuit.detector_error_model(decompose_errors=True)
matching = pymatching.Matching.from_detector_error_model(model)


In [11]:
from tqdm import tqdm
from soft_info import get_counts, soft_reweight_pymatching, counts_to_det_syndr, draw_matching_graph, reweight_edges_to_one

VERBOSE = False

#actual_observables = np.array([[False]]) # hardcoded, can be retrieved
num_errors = 0

i = 0
w_idx_lst = []
for shot in tqdm(range(len(memory))[:]):
    i += 1
    IQ_data = memory[shot]

    counts = get_counts([IQ_data], kde_dict, scaler_dict, layout, T, verbose=False)
    count_key = next(iter(counts.keys()))
    
    actual_observables = [(int(count_key[0])+1)%2]
    #soft_reweight_pymatching(matching, d, T, IQ_data, kde_dict, layout, scaler_dict, common_measure=0.01, verbose=False)  
    reweight_edges_to_one(matching)

    array_processed_string = counts_to_det_syndr(count_key, _resets=False, verbose=False)

    predicted_observables = matching.decode(array_processed_string)

    if predicted_observables == actual_observables: #== [0]:
        continue
    num_errors += 1
    #print("actual_observables:", actual_observables)
    #print("predicted_observables:", predicted_observables)
    print("num_errors:", num_errors)

    #print(f"Wrong decoding at index {i}")
    w_idx_lst.append(i)

    
    if VERBOSE:
        print("Count key:", count_key)
        print("count_string_to_syndromes:", array_processed_string)
        print("actual_observables:", actual_observables)
        print("predicted_observables:", predicted_observables)

    
    if VERBOSE:
        matched_edges = matching.decode_to_edges_array(array_processed_string)
        print("matched_edges: ", matched_edges)
        print("Estimated flip:", predicted_observables)
        
    if VERBOSE:
        draw_matching_graph(matching, d, T, syndromes=array_processed_string, matched_edges=matched_edges, figsize=(10, 10))


    

print("Num errors:", num_errors)

  7%|▋         | 672/10000 [00:48<11:07, 13.98it/s]

num_errors: 1


  7%|▋         | 714/10000 [00:51<11:12, 13.81it/s]

num_errors: 2


  8%|▊         | 842/10000 [01:00<11:12, 13.61it/s]

...

num_errors: 19


 99%|█████████▊| 9856/10000 [11:32<00:09, 14.64it/s]

num_errors: 20


100%|██████████| 10000/10000 [11:43<00:00, 14.22it/s]

Num errors: 20
